<a href="https://colab.research.google.com/github/wosmaker/CAI_6_short_story_char/blob/main/Lab6_Homework_char2index_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load pre-trained word embeddings


In [81]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip -q glove.6B.zip

**bold text**# Lab 6: Training Deep Recurrent Neural Network - Part 2

Name1, Student's ID 60070501054<br>
Name2, Student's ID 60070501055<br>

**Note: Please name your file**

## Lab Instruction - Language Modelling and Text Classification

In this lab, you will learn to train a deep recurrent neural network using LSTM with the Keras library using the Tensorflow backend. Your task is to implement the natural language modelling and text generation.

Select your favourite book from https://www.gutenberg.org/browse/scores/top and download it as a text file. Then, you will train your language model using RNN-LSTM. 

- Language model (in Thai): http://bit.ly/language_model_1
- Tutorial on how to create a language model (in English): https://medium.com/@shivambansal36/language-modelling-text-generation-using-lstms-deep-learning-for-nlp-ed36b224b275

To evaluate the model, the perplexity measurement is used: https://stats.stackexchange.com/questions/10302/what-is-perplexity

Last, fine-tune your model. You have to try different hyperparameter or adding more data. Discuss your result.



**The total lab score is 15 which will be evaluated as follows:**</br>
1. Specification (Do as the instruction said. This include the model tuning section where you have to do a proper amount of tuning) - 7 points
2. Design of logic (No weired things in the process) -  4points
3. Journaling (Communicate your thought process, comment your code, and discuss result & analyse **in every step**) - 4 points



###Import

In [82]:
import keras
from keras.preprocessing import text, sequence
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dataclasses import dataclass, field

import csv

# Moudule
import _utils as fn



###My *function*

In [83]:
class gb:
  @staticmethod
  def get_onebook(url:str, out_filename=None, postfix='.txt'):
    if out_filename is None:
      !wget $url
    else:
      name = out_filename + postfix
      !wget $url -O $name

  @staticmethod
  def get_manybook(url_list:list, prefix_name='list_book',postfix='.txt'):
    book_name = []
    for index,url in enumerate(url_list):
      name = f'{prefix_name}_{str(index)}{postfix}'
      !wget $url -O $name
      book_name.append(name)
      return book_name

**other**

In [84]:
class myfn:
  @staticmethod
  def split_text(full_text:list, maxlen:int):
    seqdata = []
    next_chars = []
    f_len = len(full_text)
    for i in range(0, f_len, maxlen):
      seqdata.append(full_text[i:i+maxlen])
      next_chars.append(full_text[min(i+maxlen, f_len-1)])
    
    return seqdata, next_chars
  
  @staticmethod
  def inv_dict(data:dict) -> dict:
    return {value:key for key, value in data.items()}
  
  def list2dict(data:list) -> dict:
    return {key:value for key, value in enumerate(data)}

###Hyper parameter

**Dataclass**

In [85]:
@dataclass
class hyper_para:
  Max_word: int = 10000
  Max_len: int = 100
  step:int = 3
  Embedding_out: int = 256
  Vocab_sizes: int = Max_word
  single_book: str = ''
  list_book: list = field(init=None)
  dir: str = '/content/drive/My Drive/LAB_6_RNN/'
  def __str__(self):
    _str = 'Hyper parameter \n\n'
    _str += f'Max_word      : {self.Max_word} \n'
    _str += f'Max_len       : {self.Max_len}  \n'
    _str += f'Step          : {self.step}  \n'
    _str += f'Embedding_out : {self.Embedding_out} \n'
    _str += f'Vocab_sizes   : {self.Vocab_sizes}  \n'
    _str += f'single_book   : {self.single_book}  \n'
    _str += f'len list_book : {len(self.list_book)} \n'
    _str += f'list_book     : {self.list_book}    \n'
    _str += f'directory     : {self.dir}    \n'

    _str += f'\n\n'
    return _str

In [86]:
list_book = ['https://www.gutenberg.org/files/84/84-0.txt',
             'https://www.gutenberg.org/files/1342/1342-0.txt',
             'https://www.gutenberg.org/files/25344/25344-0.txt',
             'https://www.gutenberg.org/files/1250/1250-0.txt',
             'https://www.gutenberg.org/files/46/46-0.txt',
             'https://www.gutenberg.org/files/1952/1952-0.txt',
             'https://www.gutenberg.org/files/16/16-0.txt'
]
hyp = hyper_para()
hyp.list_book = list_book
hyp.single_book = 'https://www.gutenberg.org/files/16/16-0.txt'
print(hyp)

Hyper parameter 

Max_word      : 10000 
Max_len       : 100  
Step          : 3  
Embedding_out : 256 
Vocab_sizes   : 10000  
single_book   : https://www.gutenberg.org/files/16/16-0.txt  
len list_book : 7 
list_book     : ['https://www.gutenberg.org/files/84/84-0.txt', 'https://www.gutenberg.org/files/1342/1342-0.txt', 'https://www.gutenberg.org/files/25344/25344-0.txt', 'https://www.gutenberg.org/files/1250/1250-0.txt', 'https://www.gutenberg.org/files/46/46-0.txt', 'https://www.gutenberg.org/files/1952/1952-0.txt', 'https://www.gutenberg.org/files/16/16-0.txt']    
directory     : /content/drive/My Drive/LAB_6_RNN/    





### 1.Load Novel FROM Gutenberge

In [87]:
# gb.get_manybook(hyp.list_book,'book')

In [88]:
# gb.get_onebook(hyp.single_book,out_filename='onebook')

### Load data from Temporary directory and Preprocessing *data*

2. Data Preprocessing 
*Note that only story will be used as a dataset, footnote and creddit are not include.*

The symbol '\n' is indicated the end of the line ``<EOS>``, which is for our model to end the sentence here.

To create a corpus for your model. The following code is can be used:</br>
*Note that other techniques can be used*

```python
# cut the text in semi-redundant sequences of maxlen characters.
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
```

The code loop through the data from first word to the last word. The maxlen define a next n word for a model to predict.


In [89]:
data_text = open("onebook.txt","r").read()
data_text = data_text.lower()
print(data_text)

chapter 1 peter breaks through

all children, except one, grow up. they soon know that they will grow
up, and the way wendy knew was this. one day when she was two years old
she was playing in a garden, and she plucked another flower and ran with
it to her mother. i suppose she must have looked rather delightful, for
mrs. darling put her hand to her heart and cried, “oh, why can't you
remain like this for ever!” this was all that passed between them on
the subject, but henceforth wendy knew that she must grow up. you always
know after you are two. two is the beginning of the end.

of course they lived at 14 [their house number on their street], and
until wendy came her mother was the chief one. she was a lovely lady,
with a romantic mind and such a sweet mocking mouth. her romantic
mind was like the tiny boxes, one within the other, that come from the
puzzling east, however many you discover there is always one more; and
her sweet mocking mouth had one kiss on it that wendy could never

**Check Number of Vocabulary and Charecter**

In [90]:
_chars = sorted(list(set(data_text)))
_word = data_text.split()
print("Total characters : ", len(_chars))
print("Total word       : ", len(_word))
print("Total Vocab      : ", len(set(_word)))

Total characters :  59
Total word       :  50532
Total Vocab      :  8809


In [91]:
_word

['chapter',
 '1',
 'peter',
 'breaks',
 'through',
 'all',
 'children,',
 'except',
 'one,',
 'grow',
 'up.',
 'they',
 'soon',
 'know',
 'that',
 'they',
 'will',
 'grow',
 'up,',
 'and',
 'the',
 'way',
 'wendy',
 'knew',
 'was',
 'this.',
 'one',
 'day',
 'when',
 'she',
 'was',
 'two',
 'years',
 'old',
 'she',
 'was',
 'playing',
 'in',
 'a',
 'garden,',
 'and',
 'she',
 'plucked',
 'another',
 'flower',
 'and',
 'ran',
 'with',
 'it',
 'to',
 'her',
 'mother.',
 'i',
 'suppose',
 'she',
 'must',
 'have',
 'looked',
 'rather',
 'delightful,',
 'for',
 'mrs.',
 'darling',
 'put',
 'her',
 'hand',
 'to',
 'her',
 'heart',
 'and',
 'cried,',
 '“oh,',
 'why',
 "can't",
 'you',
 'remain',
 'like',
 'this',
 'for',
 'ever!”',
 'this',
 'was',
 'all',
 'that',
 'passed',
 'between',
 'them',
 'on',
 'the',
 'subject,',
 'but',
 'henceforth',
 'wendy',
 'knew',
 'that',
 'she',
 'must',
 'grow',
 'up.',
 'you',
 'always',
 'know',
 'after',
 'you',
 'are',
 'two.',
 'two',
 'is',
 'the',


### Preprocessing 
### Create corpus & Word vectorization

### Preprocessing 
### maps chars to integers
chars = sorted(list(set(data)))
char_index = dict((c, i) for i, c in enumerate(chars))
index_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 100
step = 3
sentences = []
next_chars = []
for i in range(0, len(data) - maxlen, step):
    sentences.append(data[i: i + maxlen])
    next_chars.append(data[i + maxlen])


### Create corpus & Word vectorization
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        #encode features
        x[i, t, char_index[char]] = 1
        y[i, char_index[next_chars[i]]] = 1



In [92]:
print(hyp)

Hyper parameter 

Max_word      : 10000 
Max_len       : 100  
Step          : 3  
Embedding_out : 256 
Vocab_sizes   : 10000  
single_book   : https://www.gutenberg.org/files/16/16-0.txt  
len list_book : 7 
list_book     : ['https://www.gutenberg.org/files/84/84-0.txt', 'https://www.gutenberg.org/files/1342/1342-0.txt', 'https://www.gutenberg.org/files/25344/25344-0.txt', 'https://www.gutenberg.org/files/1250/1250-0.txt', 'https://www.gutenberg.org/files/46/46-0.txt', 'https://www.gutenberg.org/files/1952/1952-0.txt', 'https://www.gutenberg.org/files/16/16-0.txt']    
directory     : /content/drive/My Drive/LAB_6_RNN/    





In [93]:
hyp.step = 6
hyp.Max_word = 10000
print(hyp)

Hyper parameter 

Max_word      : 10000 
Max_len       : 100  
Step          : 6  
Embedding_out : 256 
Vocab_sizes   : 10000  
single_book   : https://www.gutenberg.org/files/16/16-0.txt  
len list_book : 7 
list_book     : ['https://www.gutenberg.org/files/84/84-0.txt', 'https://www.gutenberg.org/files/1342/1342-0.txt', 'https://www.gutenberg.org/files/25344/25344-0.txt', 'https://www.gutenberg.org/files/1250/1250-0.txt', 'https://www.gutenberg.org/files/46/46-0.txt', 'https://www.gutenberg.org/files/1952/1952-0.txt', 'https://www.gutenberg.org/files/16/16-0.txt']    
directory     : /content/drive/My Drive/LAB_6_RNN/    





In [94]:
import re
# Preprocessing
data_text = data_text.replace('\n\n', '<EOS>')

# create corpus
corpus = re.split('\n',data_text.lower())
print('block: ', len(corpus))
print('max word per block: ', max([len(x) for x in corpus]))

# clean corpus
x = [ x for x in corpus if len(x) > 0]
print(len(x))

# Create corpus & Vectorization
tokenizer = text.Tokenizer(num_words= hyp.Max_word)
tokenizer.fit_on_texts(corpus)
hyp.Vocab_sizes = len(tokenizer.word_index) + 1
print(hyp)

block:  3058
max word per block:  697
3057
Hyper parameter 

Max_word      : 10000 
Max_len       : 100  
Step          : 6  
Embedding_out : 256 
Vocab_sizes   : 5698  
single_book   : https://www.gutenberg.org/files/16/16-0.txt  
len list_book : 7 
list_book     : ['https://www.gutenberg.org/files/84/84-0.txt', 'https://www.gutenberg.org/files/1342/1342-0.txt', 'https://www.gutenberg.org/files/25344/25344-0.txt', 'https://www.gutenberg.org/files/1250/1250-0.txt', 'https://www.gutenberg.org/files/46/46-0.txt', 'https://www.gutenberg.org/files/1952/1952-0.txt', 'https://www.gutenberg.org/files/16/16-0.txt']    
directory     : /content/drive/My Drive/LAB_6_RNN/    





In [95]:
# text to sequeces
seq_vec_tmp = tokenizer.texts_to_sequences(corpus)


In [96]:
# seq_vec = []
# for items in seq_vec_tmp:
#   for i in range(0, len(items), hyp.step):
#     seq_vec.append(items[i:i+hyp.step])

seq_vec = seq_vec_tmp

print('Seq block seq_vec_tmp: ', len(seq_vec_tmp))
print('Seq block: ', len(seq_vec))
print('Seq max word per block: ', max([len(x) for x in seq_vec]))
print(f'Block of seq word more than hyperparameter({hyp.Max_len}): ', len([ x for x in seq_vec if len(x) > hyp.Max_len]), '\n')
print(f'Block of seq word min than hyperparameter(2): ', len([ x for x in seq_vec if len(x) < 2 ]), '\n')

# add pre padding
seq_vec_pad = sequence.pad_sequences(seq_vec, maxlen= hyp.Max_len, padding='pre')
print('Pad Seq block: ', len(seq_vec_pad))
print('Pad Seq max word per block: ', max([len(x) for x in seq_vec_pad]), '\n')

# check shape
print('shape of pad seq: ', seq_vec_pad.shape)

# example of seq vs pad seq
print('seq\n',seq_vec[90])
print('padseq\n',seq_vec_pad[90])

Seq block seq_vec_tmp:  3058
Seq block:  3058
Seq max word per block:  161
Block of seq word more than hyperparameter(100):  4 

Block of seq word min than hyperparameter(2):  1 

Pad Seq block:  3058
Pad Seq max word per block:  100 

shape of pad seq:  (3058, 100)
seq
 [13, 205, 11, 18, 121, 2947, 4, 2948, 2949, 141, 13, 16, 1938, 45]
padseq
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0   13  205   11   18  121 2947    4 2948 2949  141   13   16
 1938   45]


In [97]:
# example
print(tokenizer.word_index['the'])
print(tokenizer.index_word[1])

1
the


In [98]:
# create predictors and label
predictors, label = seq_vec_pad[:,:-1], seq_vec_pad[:,-1]

# One-hot label
label = keras.utils.to_categorical(label, num_classes= hyp.Vocab_sizes)

In [99]:
# show shape
print(predictors.shape, label.shape)

(3058, 99) (3058, 5698)


### Embedding matrix


In [100]:
# embeddings_index = {}
# Path = '/content/glove.6B.100d.txt'
# with open(Path) as f:
#  for line in f:
#     word, coefs = line.split(maxsplit=1)
#     coefs = np.fromstring(coefs, "f", sep=" ")
#     embeddings_index[word] = coefs

# print("Found %s word vectors." % len(embeddings_index))

In [101]:
# #Prepare embedding matrix
# hyp.Embedding_out = 100
# hits = 0
# misses = 0

# embedding_matrix = np.zeros((hyp.Vocab_sizes, hyp.Embedding_out))
# for word, i in tokenizer.word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         # Words not found in embedding index will be all-zeros.
#         # This includes the representation for "padding" and "OOV"
#         embedding_matrix[i] = embedding_vector
#         hits += 1
#     else:
#         misses += 1
# print("Converted %d words (%d misses)" % (hits, misses))

#### 3. Language Model

Define RNN model using LSTM and word embedding representation</br>
We will used perplexity as a metrics

```python
def perplexity(y_true, y_pred):
    cross_entropy = keras.backend.categorical_crossentropy(y_true, y_pred)
    perplexity = keras.backend.pow(2.0, cross_entropy)
    return perplexity
```

To used custom metrics function > https://keras.io/metrics/

For a loss function `categorical_crossentropy` is used, any optimzation method can be applied.

In [113]:
from keras.layers import Embedding, Input, Bidirectional
from keras.layers import LSTM, GRU
from keras.layers import Dropout 
from keras.layers import Dense
import keras.backend 
from keras.callbacks import ModelCheckpoint

In [103]:
def perplexity(y_true, y_pred):
    cross_entropy = keras.backend.categorical_crossentropy(y_true, y_pred)
    perplexity = keras.backend.pow(2.0, cross_entropy)
    return perplexity

In [104]:
print(predictors.shape, label.shape)

(3058, 99) (3058, 5698)


In [105]:
# embedding_layer = Embedding(
#     hyp.Vocab_sizes,
#     hyp.Embedding_out,
#     embeddings_initializer=keras.initializers.Constant(embedding_matrix),
#     trainable=False,
# )

In [106]:
# # Define your model

# seq_input = Input(shape=(None,), dtype='int32')
# emb_seq = embedding_layer(seq_input)
# x = GRU(256, return_sequences=False)(emb_seq)
# x = Dense(hyp.Vocab_sizes // 6, activation='elu')(x)
# preds = Dense(hyp.Vocab_sizes, activation='softmax')(x)

# model = keras.Model(seq_input, preds)
# model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=[perplexity])

# print(model.summary())

In [107]:
# check point
FilePath = hyp.dir + 'model_checkpoint_08.h5'
checkpoint = ModelCheckpoint(FilePath)

In [108]:
#load model


In [117]:
# Define your model

model = keras.models.Sequential()

model.add(Embedding(hyp.Vocab_sizes, 256,input_length=hyp.Max_len-1))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(LSTM(256, return_sequences=False))
model.add(Dense(hyp.Vocab_sizes // 4, activation='elu'))
model.add(Dense(hyp.Vocab_sizes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=[perplexity])
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 99, 256)           1458688   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 99, 512)           1050624   
_________________________________________________________________
lstm_7 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense_15 (Dense)             (None, 1424)              365968    
_________________________________________________________________
dense_16 (Dense)             (None, 5698)              8119650   
Total params: 11,782,386
Trainable params: 11,782,386
Non-trainable params: 0
_________________________________________________________________
None


In [118]:
# Training your model
history = model.fit(predictors, label, epochs=30, batch_size=32, verbose=1, callbacks=[checkpoint])

Epoch 1/30
96/96 [==============================] - 5s 50ms/step - loss: 6.9637 - perplexity: 382.3561
Epoch 2/30
96/96 [==============================] - 5s 47ms/step - loss: 5.9776 - perplexity: 155.7220
Epoch 3/30
96/96 [==============================] - 5s 48ms/step - loss: 5.7585 - perplexity: 123.7573
Epoch 4/30
96/96 [==============================] - 5s 47ms/step - loss: 5.5635 - perplexity: 98.6615
Epoch 5/30
96/96 [==============================] - 5s 47ms/step - loss: 5.4104 - perplexity: 85.6462
Epoch 6/30
96/96 [==============================] - 4s 47ms/step - loss: 5.2467 - perplexity: 75.2111
Epoch 7/30
96/96 [==============================] - 4s 47ms/step - loss: 5.0554 - perplexity: 68.7095
Epoch 8/30
96/96 [==============================] - 5s 47ms/step - loss: 4.8745 - perplexity: 61.2247
Epoch 9/30
96/96 [==============================] - 5s 47ms/step - loss: 4.6497 - perplexity: 54.4781
Epoch 10/30
96/96 [==============================] - 4s 47ms/step - loss: 4.441

In [37]:
model.save(hyp.dir + 'model_back_08.h5')

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### 4. Evaluate your model 

In [39]:
# Evaluate your model using perplexity measurment (You can try adding other measurements as well)

#### 5. Text generating

In [119]:
def generate_text(seed_text:str, model:model, tokenizer:tokenizer, hyp:hyper_para, maxlen:int = 200):
    # Loop through the next n words

    _token_nopad = tokenizer.texts_to_sequences([seed_text])
    _token = sequence.pad_sequences(_token_nopad,maxlen=hyp.Max_len,padding='pre')[0]

    str_memory = seed_text

    for _ in range(maxlen):
      _token = model.predict_classes(_token)
      str_memory += ' ' + tokenizer.sequences_to_texts([_token])[0]

    return str_memory

In [120]:
seed_text = 'i was'
string = generate_text(seed_text, model, tokenizer, hyp, maxlen=200)
string

'i was take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take take was is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is practical practical practical practical practical practical practical practical practical practical practical practical practical practical practical practical practical practical practical practical

In [72]:
model.reset_states()

In [76]:
seed_text = 'i was you are'
_token_nopad = tokenizer.texts_to_sequences([seed_text])
_token = sequence.pad_sequences(_token_nopad,maxlen=hyp.Max_len,padding='pre')[0]
_token = model.predict_classes(_token)
str_memory = seed_text
str_memory += ' ' + tokenizer.sequences_to_texts([_token])[0]

str_memory

'i was you are you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you thought sitting must scattered'

In [ ]:
seed_text = "i"
_s = []
print('test',_s)
a = tokenizer.texts_to_sequences([seed_text])
print(a)
b = sequence.pad_sequences(a, maxlen=hyp.Max_len,padding='pre')[0]
len(b)

In [ ]:
model.reset_states()
c = model.predict(b)
c.shape

In [ ]:
c[0]

In [ ]:
max(c[10])

In [ ]:
tokenizer.sequences_to_texts([c])

In [ ]:
model.predict_classes(a)

In [ ]:
tokenizer.index_word[1]

In [ ]:
a = [1,2,3,5,6]
a = 
a

In [ ]:
# generate your sample text

#### 6. Model Tuning 

Write down why you design this architecture or why you choose this set of parameter</br>
You should have at least 1 different architectures/set of hyperparameters per person in your team</br>
Last, train your best performed model **on 30 epoch** (or you can try 50+ epoch but this will take time)</br>
*Note: For the last step, please turn off a verbose during training

In [ ]:
# Try out different hyperparameter & model architecture

####[Speacial] 7. Help your model to generate a short story 

**Example** https://medium.com/deep-writing/harry-potter-written-by-artificial-intelligence-8a9431803da6

Write your result in a `markdown` cell

In [ ]:
# Create your short-story from your model (Add your creativity here)

### More on Natural language Processing and Language model
1. https://medium.com/@ageitgey/natural-language-processing-is-fun-9a0bff37854e 
2. https://medium.com/phrasee/neural-text-generation-generating-text-using-conditional-language-models-a37b69c7cd4b
3. http://karpathy.github.io/2015/05/21/rnn-effectiveness/

**Music generates by RNN**
https://soundcloud.com/optometrist-prime/recurrence-music-written-by-a-recurrent-neural-network
